In [54]:
import os
import pandas as pd
import numpy as np
from scipy.stats import iqr
from statsmodels.formula.api import ols
from scipy.stats import zscore
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import pandas as pd
import scipy.stats as stats
from statsmodels.stats.multitest import multipletests
# Create bar plot
plt.style.use('seaborn-v0_8-poster')
#plt.style.use('seaborn-v0_8')
fontsize = 11
plt.rcParams.update({
    "font.size": fontsize,          # Global font size
    "axes.labelsize": fontsize,     # X and Y axis labels
    "axes.titlesize": fontsize,     # Title (if used)
    "xtick.labelsize": fontsize,    # X-axis tick labels
    "ytick.labelsize": fontsize,    # Y-axis tick labels
    "legend.fontsize": fontsize,    # Legend font size
})

In [55]:
project_dir = '/Users/xiaoqianxiao/projects/NARSAD'
SCR_dir = os.path.join(project_dir, 'EDR')
fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')
plot_dir = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/plots/secondHalf')

In [56]:
def rename_roi(roi):
    cleaned_roi = roi.replace('_mask', '').replace('_mask', '')
    sides = ['lh', 'rh']
    regions = ['hippocampus', 'amygdala', 'vmpfc', 'insula', 'ACC']  # Add more as needed
    
    side = None
    region = None
    for s in sides:
        if s in cleaned_roi:
            side = s
            break
    for r in regions:
        if r in cleaned_roi:
            region = r
            break
    if side and region:
        return f"{side}_{region}"
    else:
        return cleaned_roi

In [57]:
# Define directories
root_dir = '/Users/xiaoqianxiao/projects'
project_name = 'NARSAD'

sub_no_MRI_phase2 = ['N102', 'N208']
sub_no_MRI_phase3 = ['N102', 'N208', 'N120']

SCR_dir = os.path.join(root_dir, project_name, 'EDR')
drug_file = os.path.join(SCR_dir, 'drug_order.csv')
ECR_file = os.path.join(SCR_dir, 'ECR.csv')

# Load behavioral data
df_drug = pd.read_csv(drug_file)
df_drug['group'] = df_drug['subID'].apply(lambda x: 'Patients' if x.startswith('N1') else 'Controls')
df_ECR = pd.read_csv(ECR_file)
df_behav = df_drug.merge(df_ECR, how='left', left_on='subID', right_on='subID')

group_info_df_phase2 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase2)]
group_info_df_phase3 = df_behav.loc[~df_behav['subID'].isin(sub_no_MRI_phase3)]

fMRI_file = os.path.join(project_dir, 'MRI/derivatives/fMRI_analysis/groupLevel/roi_cope_results_all.csv')
df_fMRI = pd.read_csv(fMRI_file)
df_fMRI['roi_name'] = df_fMRI['ROI'].apply(rename_roi)
# Define the desired order for 'roi_name'
roi_order = ['lh_ACC', 'rh_ACC', 
             'lh_vmpfc','rh_vmpfc',
             'lh_insula','rh_insula', 
             'lh_hippocampus', 'rh_hippocampus', 
             'lh_amygdala', 'rh_amygdala'
]

df_fMRI['roi_name'] = pd.Categorical(df_fMRI['roi_name'], categories=roi_order, ordered=True)

df_fMRI_phase2 = df_fMRI.loc[df_fMRI['Task']=='phase2'].copy()
df_fMRI_phase3 = df_fMRI.loc[df_fMRI['Task']=='phase3'].copy()

In [58]:
df_fMRI_phase2['subID'] = df_fMRI_phase2['Subject'].apply(lambda x: group_info_df_phase2.iloc[x - 1]['subID'])
df_fMRI_phase3['subID'] = df_fMRI_phase3['Subject'].apply(lambda x: group_info_df_phase3.iloc[x - 1]['subID'])
df_phase2 = df_fMRI_phase2.merge(group_info_df_phase2, how='left', left_on='subID', right_on='subID')
df_phase2 = df_phase2[df_phase2['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)
df_phase3 = df_fMRI_phase3.merge(group_info_df_phase3, how='left', left_on='subID', right_on='subID')
df_phase3 = df_phase3[df_phase3['Gender'].isin(['Female', 'Male'])].reset_index(drop=True)

In [59]:
df_current = df_phase2

In [60]:
results = []
from scipy.stats import ttest_rel
for roi in df_current['roi_name'].unique():
    for group in df_current['group'].unique():
        df_tmp = df_current.loc[(df_current['roi_name'] == roi) &
                                (df_current['group'] == group) &
                                (df_current['Contrast'].isin([20,22]))].copy()
        df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
        t_statistic, p_value = stats.ttest_rel(df_pivot[[20]], df_pivot[[22]])
        results.append({
            'roi': roi,
            'group': group,
            't_stat': t_statistic[0],
            'p_value': p_value[0],
            'n': len(df_pivot)  # Sample size
        })
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

,roi,group,t_stat,p_value,n,p_value_corrected
11,lh_insula,Controls,3.408330,0.001284,52,0.025683
8,rh_vmpfc,Patients,-2.420750,0.019321,49,0.386410
0,lh_hippocampus,Patients,-0.376940,0.707880,49,1.000000
17,lh_vmpfc,Controls,0.886023,0.379764,52,1.000000
16,lh_vmpfc,Patients,-1.272680,0.209262,49,1.000000
15,rh_insula,Controls,-0.319641,0.750546,52,1.000000
14,rh_insula,Patients,-0.087016,0.931021,49,1.000000
13,rh_amygdala,Controls,-0.247088,0.805832,52,1.000000
12,rh_amygdala,Patients,-0.604767,0.548182,49,1.000000
10,lh_insula,Patients,-0.049660,0.960599,49,1.000000


In [61]:
results = []
from scipy.stats import ttest_rel
for roi in df_current['roi_name'].unique():
    for group in df_current['group'].unique():
        for gender in df_current['Gender'].unique():
            for drug in df_current['Drug'].unique():
                df_tmp = df_current.loc[(df_current['roi_name'] == roi) & 
                                        (df_current['group'] == group) & 
                                        (df_current['Gender'] == gender) &
                                        (df_current['Drug'] == drug) &
                                        (df_current['Contrast'].isin([20,22]))].copy()
                df_pivot = df_tmp.pivot(index='Subject', columns='Contrast', values='Mean_COPE_Value')
                t_statistic, p_value = stats.ttest_rel(df_pivot[[20]], df_pivot[[22]])
                results.append({
                'roi': roi,
                'Group': group,
                'Drug': drug,
                'Gender': gender,
                't_stat': t_statistic[0],
                'p_value': p_value[0],
                'n': len(df_pivot)  # Sample size
            })
df_results = pd.DataFrame(results)
df_results['p_value_corrected'] = multipletests(df_results['p_value'], method='bonferroni')[1]
df_results.sort_values(by='p_value_corrected', ascending=True)

,roi,Group,Drug,Gender,t_stat,p_value,n,p_value_corrected
12,rh_ACC,Controls,Placebo,Female,3.960186,0.001630,14,0.130393
44,lh_insula,Controls,Placebo,Female,3.520422,0.003764,14,0.301102
0,lh_hippocampus,Patients,Placebo,Female,-0.639268,0.534661,13,1.000000
57,rh_insula,Patients,Oxytocin,Female,-1.367388,0.191645,16,1.000000
56,rh_insula,Patients,Placebo,Female,1.193361,0.255788,13,1.000000
...,...,...,...,...,...,...,...,...
24,lh_ACC,Patients,Placebo,Female,-0.077634,0.939399,13,1.000000
23,lh_amygdala,Controls,Oxytocin,Male,-0.281743,0.783370,12,1.000000
22,lh_amygdala,Controls,Placebo,Male,-0.450010,0.660116,14,1.000000
29,lh_ACC,Controls,Oxytocin,Female,2.259730,0.045113,12,1.000000


In [62]:
# extinction
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    # contrast no.2: CS+_safe - fixation; contrast no.3: CS+_reinf - fixation;no.4: CS+_safe - CS-; contrast no.6: CS+_reinf - CS-;
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin([20,22]))].copy()
    df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    combined_col = f'{roi}'
        
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'Condition': combined_col,
            'Effect': effect_idx,
            'p_value': p_value
        })
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
    for effect_idx in result_ols.params.index:
        if effect_idx == 'C(Drug)[T.Placebo]': 
            drug_results_list.append({
                'Effect': effect_idx,
                'ROI': roi,
                'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
            })

# Create DataFrame for drug effects
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('ROI')
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.090
Model:                            OLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     1.001
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.445
Time:                        13:06:06   Log-Likelihood:                -138.54
No. Observations:                 101   AIC:                             297.1
Df Residuals:                      91   BIC:                             323.2
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                                                                coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------

,Condition,Effect,p_value
68,rh_amygdala,Avoidance_centered,0.004140
88,lh_vmpfc,Avoidance_centered,0.024312
94,rh_hippocampus,C(group)[T.Patients]:C(Drug)[T.Placebo],0.041911
33,lh_ACC,C(Gender)[T.Male],0.044279
7,lh_hippocampus,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.048430
...,...,...,...
86,lh_vmpfc,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.931158
26,lh_amygdala,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.972263
47,rh_vmpfc,C(group)[T.Patients]:C(Drug)[T.Placebo]:C(Gend...,0.985323
93,rh_hippocampus,C(Gender)[T.Male],0.986964


for the entire dataset \
avoidance in rh-amy and lh-vmpfc \
group * drug in rh-hip \
gender in lh_acc \
group * drug * gender in lh-hip

In [63]:
# extinction in only placebo
results_list = []
drug_results_list = []
for roi in df_current['roi_name'].unique():
    print('extinction results for {}'.format(roi))
    df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['roi_name'] == roi) & (df_current['Contrast'].isin([20,22]))].copy()
    df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
    df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
    df['fMRI'] = df['CSR'] - df['CSS']
    df['z_fMRI'] = 1 + zscore(df['fMRI'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
    
    df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
    
    model_ols = ols("z_fMRI ~ C(group) * C(Gender) + Avoidance_centered + demo_age", data=df)
    result_ols = model_ols.fit()
    print(result_ols.summary())
    conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI     
    for effect_idx in result_ols.pvalues.index:
        p_value = result_ols.pvalues[effect_idx]
        results_list.append({
            'ROI': roi,
            'Effect': effect_idx,
            'b': result_ols.params[effect_idx],  # Coefficient
                'SE': result_ols.bse[effect_idx],    # Standard error
                't': result_ols.tvalues[effect_idx], # t-statistic
                'p': result_ols.pvalues[effect_idx],  # p-value
                'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                'CI_upper': conf_int.loc[effect_idx, 1]
        })
    
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p', ascending=True)

extinction results for lh_hippocampus
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.076
Model:                            OLS   Adj. R-squared:                 -0.029
Method:                 Least Squares   F-statistic:                    0.7269
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.607
Time:                        13:08:39   Log-Likelihood:                -68.963
No. Observations:                  50   AIC:                             149.9
Df Residuals:                      44   BIC:                             161.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------

,ROI,Effect,b,SE,t,p,CI_lower,CI_upper
17,lh_amygdala,demo_age,0.057440,0.021107,2.721364,0.009279,0.014901,0.099978
8,rh_ACC,C(Gender)[T.Male],0.948356,0.369837,2.564255,0.013830,0.202999,1.693712
7,rh_ACC,C(group)[T.Patients],0.948933,0.393354,2.412413,0.020083,0.156179,1.741687
25,rh_vmpfc,C(group)[T.Patients],0.934333,0.395830,2.360439,0.022751,0.136590,1.732077
40,rh_amygdala,Avoidance_centered,0.015364,0.007396,2.077444,0.043629,0.000459,0.030269
20,lh_ACC,C(Gender)[T.Male],0.749774,0.398721,1.880445,0.066677,-0.053797,1.553344
55,rh_hippocampus,C(group)[T.Patients],0.699649,0.400558,1.746686,0.087670,-0.107623,1.506920
59,rh_hippocampus,demo_age,0.034755,0.020974,1.657041,0.104626,-0.007516,0.077027
1,lh_hippocampus,C(group)[T.Patients],0.705744,0.428841,1.645701,0.106951,-0.158528,1.570016
9,rh_ACC,C(group)[T.Patients]:C(Gender)[T.Male],-0.802398,0.557853,-1.438369,0.157405,-1.926677,0.321881


In placebo. \
Group: rh-acc, rh_vmpfc \
Gender: rh-ACC \
Age: lh-amy \
avoidance: rh-amy

In [64]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin([20,22]))].copy()
df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                col='group',
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_group.png', bbox_inches='tight')
plt.close()
#plt.show()

In [65]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin([20,22]))].copy()
df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='Gender', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_gender_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [66]:
# placebo data, gender * group
df_tmp = df_current[(df_current['Drug'] == 'Placebo') & (df_current['Contrast'].isin([20,22]))].copy()
df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
df = df_tmp.copy()
df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()

df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')

plt.style.use('seaborn-v0_8-poster')

g = sns.catplot(x='group', y='z_fMRI', hue='CS_Type',
                row='roi_name', 
                kind='bar', 
                hue_order=['CSS', 'CSR'],
                errorbar='se',
                height=6, aspect=1,
                data=df)

g.set_axis_labels('', 'z_fMRI')
#g.legend.set_title('')
  
y_min = df['z_fMRI'].min() - 0.5
y_max = df['z_fMRI'].max() + 0.5
g.set(ylim=(0, 2.2))

g.figure.suptitle('', fontsize=22, y=1.01)  # Add title above plot

g.figure.set_dpi(300)
plt.savefig(f'{plot_dir}/extinction_placebodata_group_effect.png', bbox_inches='tight')
plt.close()
#plt.show()

In [67]:
# extinction in each group
results_list = []
drug_results_list = []
for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {group}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                          (df_current['group'] == group) & 
                          (df_current['Contrast'].isin([20,22]))].copy()
        
        df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                        columns='CS_Type', 
                        values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(Drug) * C(Gender) + Avoidance_centered + demo_age", data=df)
        #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        
        combined_col = f'{roi}_{group}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Effect': effect_idx,
                'p_value': p_value
            })
        conf_int = result_ols.conf_int(alpha=0.05)  # 95% CI 
        for effect_idx in result_ols.params.index:
            if effect_idx == 'C(Drug)[T.Placebo]': 
                drug_results_list.append({
                    'Effect': effect_idx,
                    'group': group,
                    'ROI': roi,
                    'b': result_ols.params[effect_idx],  # Coefficient
                    'SE': result_ols.bse[effect_idx],    # Standard error
                    't': result_ols.tvalues[effect_idx], # t-statistic
                    'p': result_ols.pvalues[effect_idx],  # p-value
                    'CI_lower': conf_int.loc[effect_idx, 0],  # Lower bound of 95% CI
                    'CI_upper': conf_int.loc[effect_idx, 1]
                })
    
df_drug = pd.DataFrame(drug_results_list)
df_drug = df_drug.sort_values('p', ascending=True)
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Patients
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.125
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1.225
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.314
Time:                        13:10:23   Log-Likelihood:                -66.266
No. Observations:                  49   AIC:                             144.5
Df Residuals:                      43   BIC:                             155.9
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------

,Condition,Effect,p_value
40,rh_amygdala_Patients,Avoidance_centered,0.006522
52,lh_vmpfc_Patients,Avoidance_centered,0.008625
119,rh_hippocampus_Controls,demo_age,0.034911
43,rh_insula_Patients,C(Drug)[T.Placebo],0.035201
57,rh_hippocampus_Patients,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.042994
...,...,...,...
87,rh_vmpfc_Controls,C(Drug)[T.Placebo]:C(Gender)[T.Male],0.976513
73,lh_amygdala_Controls,C(Drug)[T.Placebo],0.978135
88,rh_vmpfc_Controls,Avoidance_centered,0.990873
31,lh_insula_Patients,C(Drug)[T.Placebo],0.996349


in group specific dataset, \
for Patients: \
    avoidance in rh-amy, lh-vmpfc \
    drug in rh-insula \
    drug * gender in rh-hip

for controls: \
    age in rh-hip \ 

In [68]:
import pandas as pd
from scipy.stats import zscore, ttest_ind

desired_comparisons = [
    'Female_Oxytocin vs. Female_Placebo',
    'Male_Oxytocin vs. Male_Placebo'
]

t_test_results_list = []

for group in df_current['group'].unique():
    for roi in df_current['roi_name'].unique():
        df_tmp = df_current[(df_current['roi_name'] == roi) & 
                           (df_current['group'] == group) & 
                           (df_current['Contrast'].isin([20, 22]))].copy()
        
        df_tmp.loc[df_tmp['Contrast'] == 20, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast'] == 22, 'CS_Type'] = 'CSR'
        
        df = df_tmp.pivot(index=['subID', 'Drug', 'Gender', 'group', 'Avoidance', 'demo_age'], 
                          columns='CS_Type', values='Mean_COPE_Value').reset_index()
        
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])  # Offset of 1 assumed intentional
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
        
        group_labels = df['Gender'].astype(str) + "_" + df['Drug'].astype(str)
        
        for comparison in desired_comparisons:
            g1, g2 = comparison.split(' vs. ')
            if g1 in group_labels.values and g2 in group_labels.values:  # Check if both groups exist
                group1_data = df[group_labels == g1]['z_fMRI']
                group2_data = df[group_labels == g2]['z_fMRI']
                
                t_stat, p_value = ttest_ind(group1_data, group2_data, nan_policy='omit')
                
                mean_diff = group1_data.mean() - group2_data.mean()
                se = ((group1_data.var() / group1_data.count()) + (group2_data.var() / group2_data.count())) ** 0.5
                
                t_test_results_list.append({
                    'ROI': roi,
                    'Comparison': comparison,
                    'Mean_Diff': mean_diff,
                    'SE': se,
                    't': t_stat,
                    'p': p_value
                })

# Create DataFrame with all results
df_t_test = pd.DataFrame(t_test_results_list)
df_t_test = df_t_test.sort_values('p', ascending=True)  # Sort by p-value

In [69]:
df_t_test

,ROI,Comparison,Mean_Diff,SE,t,p
18,rh_hippocampus,Female_Oxytocin vs. Female_Placebo,-0.840911,0.301716,-2.674584,0.012549
22,rh_ACC,Female_Oxytocin vs. Female_Placebo,0.674049,0.366181,1.866382,0.074254
14,rh_insula,Female_Oxytocin vs. Female_Placebo,0.684082,0.378222,1.734944,0.094154
1,lh_hippocampus,Male_Oxytocin vs. Male_Placebo,0.758083,0.479572,1.519458,0.146017
29,rh_vmpfc,Male_Oxytocin vs. Male_Placebo,0.485017,0.353338,1.406878,0.172282
20,lh_hippocampus,Female_Oxytocin vs. Female_Placebo,0.508666,0.370090,1.342720,0.191925
31,lh_insula,Male_Oxytocin vs. Male_Placebo,0.578419,0.440040,1.325380,0.197525
33,rh_amygdala,Male_Oxytocin vs. Male_Placebo,-0.449738,0.355465,-1.222335,0.233450
21,lh_hippocampus,Male_Oxytocin vs. Male_Placebo,-0.475329,0.408577,-1.147737,0.262390
28,rh_vmpfc,Female_Oxytocin vs. Female_Placebo,0.485915,0.425944,1.147239,0.262592


In [70]:
results_list = []
# extinction, in each gender
for g in df_current['Gender'].unique():
    for roi in df_current['roi_name'].unique():
        print(f'extinction results for {roi} {g}')
        df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Gender'] == g) & (df_current['Contrast'].isin([20,22]))].copy()
        df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
        df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
        df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], columns='CS_Type', values='Mean_COPE_Value').reset_index()
        df['fMRI'] = df['CSR'] - df['CSS']
        df['z_fMRI'] = 1 + zscore(df['fMRI'])
        df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
        df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
        
        df[['group', 'Gender','drug']] = df[['group', 'Gender' ,'Drug']].astype('category')
        
        model_ols = ols("z_fMRI ~ C(group) * C(Drug) + Avoidance_centered + demo_age", data=df)
        result_ols = model_ols.fit()
        print(result_ols.summary())
        combined_col = f'{roi}_{g}'
        
        for effect_idx in result_ols.pvalues.index:
            p_value = result_ols.pvalues[effect_idx]
            results_list.append({
                'Condition': combined_col,
                'Gender': g,
                'Effect': effect_idx,
                'p_value': p_value
            })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.043
Method:                 Least Squares   F-statistic:                     1.482
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.213
Time:                        13:12:28   Log-Likelihood:                -74.169
No. Observations:                  55   AIC:                             160.3
Df Residuals:                      49   BIC:                             172.4
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------

,Condition,Gender,Effect,p_value
100,rh_amygdala_Male,Male,Avoidance_centered,0.005775
52,lh_vmpfc_Female,Female,Avoidance_centered,0.010491
59,rh_hippocampus_Female,Female,demo_age,0.011174
57,rh_hippocampus_Female,Female,C(group)[T.Patients]:C(Drug)[T.Placebo],0.027804
99,rh_amygdala_Male,Male,C(group)[T.Patients]:C(Drug)[T.Placebo],0.051115
...,...,...,...,...
16,lh_amygdala_Female,Female,Avoidance_centered,0.922817
71,rh_ACC_Male,Male,demo_age,0.949982
88,rh_vmpfc_Male,Male,Avoidance_centered,0.950160
34,lh_insula_Female,Female,Avoidance_centered,0.963444


in gender specific dataset:\
For female:\
avoidance in lh-vmpfc\
age in rh-hip\
group * drug in rh-hip\
For male:\
avoidance in rh-amy
group * drug in rh-amy \

In [71]:
import pandas as pd 
from statsmodels.formula.api import ols
from scipy.stats import zscore

results_list = []
for g in df_current['Gender'].unique():
    for group in df_current['group'].unique():
        for roi in df_current['roi_name'].unique():
            print(f'extinction results for {roi} {g}')
            df_tmp = df_current[(df_current['roi_name'] == roi) & 
                              (df_current['Gender'] == g) & 
                              (df_current['group'] == group) & 
                              (df_current['Contrast'].isin([20,22]))].copy()
            
            df_tmp.loc[df_tmp['Contrast']==20, 'CS_Type'] = 'CSS'
            df_tmp.loc[df_tmp['Contrast']==22, 'CS_Type'] = 'CSR'
            
            df = df_tmp.pivot(index=['subID','Drug','Gender','group','Avoidance','demo_age'], 
                            columns='CS_Type', 
                            values='Mean_COPE_Value').reset_index()
            
            df['fMRI'] = df['CSR'] - df['CSS']
            df['z_fMRI'] = 1 + zscore(df['fMRI'])
            df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
            df['age_centered'] = df['demo_age'] - df['demo_age'].mean()
            
            df[['group', 'Gender','Drug']] = df[['group', 'Gender','Drug']].astype('category')
            
            model_ols = ols("z_fMRI ~ C(Drug) + Avoidance_centered + demo_age", data=df)
            
            #model_ols = ols("z_fMRI ~ C(Drug)", data=df)
            result_ols = model_ols.fit()
            print(result_ols.summary())
            
            combined_col = f'{roi}_{group}_{g}'
            
            for effect_idx in result_ols.pvalues.index:
                p_value = result_ols.pvalues[effect_idx]
                results_list.append({
                    'Gender': g,
                    'Group': group,
                    'Condition': combined_col,
                    'Effect': effect_idx,
                    'p_value': p_value
                })
results_df = pd.DataFrame(results_list)
results_df.loc[results_df['Effect']!='Intercept'].sort_values('p_value', ascending=True)

extinction results for lh_hippocampus Female
                            OLS Regression Results                            
Dep. Variable:                 z_fMRI   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                 -0.118
Method:                 Least Squares   F-statistic:                   0.01271
Date:                Thu, 20 Mar 2025   Prob (F-statistic):              0.998
Time:                        13:12:42   Log-Likelihood:                -41.127
No. Observations:                  29   AIC:                             90.25
Df Residuals:                      25   BIC:                             95.72
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------

,Gender,Group,Condition,Effect,p_value
106,Male,Patients,rh_amygdala_Patients_Male,Avoidance_centered,0.010486
91,Male,Patients,lh_amygdala_Patients_Male,demo_age,0.013487
119,Male,Patients,rh_hippocampus_Patients_Male,demo_age,0.015272
34,Female,Patients,lh_vmpfc_Patients_Female,Avoidance_centered,0.021313
99,Male,Patients,rh_vmpfc_Patients_Male,demo_age,0.027383
...,...,...,...,...,...
98,Male,Patients,rh_vmpfc_Patients_Male,Avoidance_centered,0.971112
138,Male,Controls,rh_vmpfc_Controls_Male,Avoidance_centered,0.986054
25,Female,Patients,rh_amygdala_Patients_Female,C(Drug)[T.Placebo],0.994457
35,Female,Patients,lh_vmpfc_Patients_Female,demo_age,0.996203


in group and gender specific dataset:\
drug effect was found for patients in rh-hip for females\
and rh-amy and lh-hip for males\


In [72]:
for roi in df_current['roi_name'].unique():
    print(f'{roi}')
    df_tmp = df_current[(df_current['roi_name'] == roi) & (df_current['Contrast'].isin([20, 22]))].copy()
    df_tmp.loc[df_tmp['Contrast'] == 20, 'CS_Type'] = 'CSS'
    df_tmp.loc[df_tmp['Contrast'] == 22, 'CS_Type'] = 'CSR'
    df = df_tmp.copy()
    df['z_fMRI'] = 1 + zscore(df['Mean_COPE_Value'])
    df['Avoidance_centered'] = df['Avoidance'] - df['Avoidance'].mean()
    
    df[['group', 'Gender', 'Drug']] = df[['group', 'Gender', 'Drug']].astype('category')
    
    plt.style.use('seaborn-v0_8-poster')
    
    g = sns.catplot(x='Drug', y='z_fMRI', hue='CS_Type',
                    order=['Placebo', 'Oxytocin'], 
                    col='group', 
                    row='Gender', 
                    kind='bar', 
                    hue_order=['CSS', 'CSR'],
                    errorbar='se',
                    height=6, aspect=1,
                    data=df)
    
    g.set_axis_labels('', 'z_fMRI')
    #g.legend.set_title('')
    if g.legend is not None:
        g.legend.set_title('')
    else:
        print(f"Warning: No legend generated for {roi_name}")    
    y_min = df['z_fMRI'].min() - 0.5
    y_max = df['z_fMRI'].max() + 0.5
    g.set(ylim=(0, 2.2))
    
    g.figure.suptitle(f'{roi} in Extinction', fontsize=22, y=1.01)  # Add title above plot
    
    g.figure.set_dpi(300)
    plt.savefig(f'{plot_dir}/extinction_plot_{roi}.png', bbox_inches='tight')
    #plt.show()
    plt.close()

lh_hippocampus
rh_ACC
lh_amygdala
lh_ACC
rh_vmpfc
lh_insula
rh_amygdala
rh_insula
lh_vmpfc
rh_hippocampus
